|First Name     |Last Name    |MtkNr.      |Due Date  | 
|:-------------:|:-----------:|:----------:|:--------:|
| Giovanni    | Filomeno  |k12315325     | 09.12.2024, 16:00

<h1 style="color:rgb(150,100,10)">Computer Vision Course</h1>
<h2 style="color:rgb(150,100,10)">Assignment 3 – Optical Flow, Object Tracking and Detection</h2>

<b>Authors:</b> O. Bimber, M. Abbass<br>
<b>Date:</b> 25-09-2024

<b>This file is part of the "Computer Vision Course 2024W" UE material.</b>

<h2 style="color:rgb(150,100,10)">Table of Contents</h2>
<ol>
    <a style="color:rgb(150,100,10)" href="#Optical-Flow"><li style="font-size:large;font-weight:bold">Optical Flow</li></a>
    <ol style="margin-bottom:15px">
        <a style="color:rgb(150,100,10)" href="#Apply-Lucas-Kanade"><li style="font-size:medium">Apply Lucas-Kanade</li></a>
        <a style="color:rgb(150,100,10)" href="#Apply-Farneback"><li style="font-size:medium">Apply Farneback Optical Flow</li></a>
</ol>
    <a style="color:rgb(150,100,10)" href="#Object-Tracking"><li style="font-size:large;font-weight:bold">Object Tracking</li></a>
    <ol style="margin-bottom:15px">
        <a style="color:rgb(150,100,10)" href="#Apply-Yolov8-for-object-tracking"><li style="font-size:medium">Apply Yolov8 for object tracking</li></a>
</ol></a>
    </ol>
</ol>


<h1 style="color:rgb(255,0,0)">Important Note</h1>

<b>In this assignment, you can use the default parameters in any built-in function unless specified otherwise.</b>

<b>All cells must be evaluated, and any unevaluated cell may lead to a loss of points, regardless of the correctness of the code.</b>

<b>If you have GPU resources available on your machine, please utilize them; otherwise, you can use your CPU.</b>

In [12]:
# Import all packages needed in this notebook.
import numpy as np
import cv2 as cv
from PIL import Image
from matplotlib import pyplot as plt
import os
import glob
from scipy import signal
import shutil

if not os.path.exists('resources'): raise TypeError("Please create the resources folder and include all required stuff")

<a name="Optical-Flow"></a><h2> Optical Flow</h2>
    
The main goal is to utilize optical flow to track object features. Two general optical flow approaches exist: Sparse Optical Flow (e.g., <i><b>the Lucas-Kanade method</b></i>) and Dense Optical Flow (e.g., <i><b>the Farneback method</b></i>).

<a name="Apply-Lucas-Kanade"></a><h3>Apply Lucas-Kanade</h3>

<b>Exercise 1.1. [20 Points]</b>
   
<b>Goal:</b> The aim here is to apply the <i><b>Lucas-Kanade Method</b></i>.

<b>Task</b>
<ul>
<li>Complete the function <i><b>sparse_optical</b></i> below to compute sparse optical flow</li>
<li>The <i><b>Lucas-Kanade Method</b></i> can be used from the OpenCV library.</li>
<li>The parameters of the <i><b>Lucas-Kanade Method</b></i> are as follows: 
<ol>
  <li>Use the <i><b>good_Feture</b></i> variable as an input flow vector</li>
  <li>Size of the search window at each pyramid level = 15</li>
  <li>Maximal pyramid level number = 2</li>
  <li>Criteria = 10, 0.03</li>
  <li>output vector = <i><b>None</b></i></li>
  <li>output status vector = <i><b>None</b></i></li>
  <li>output vector of errors = <i><b>None</b></i></li>
</ol>


</ul>

<b>Hint</b>
<ul>     
<li>The function <i><b>sparse_optical</b></i> uses the video <i><b>vtest.avi</b></i></li> 
<li>The video <i><b>vtest.avi</b></i> is in the <i><b>resources</b></i> folder. <i><b>Therefore, you must make sure to include this video.</b></i>.</li>
    
</ul>

In [13]:
def sparse_optical(path_Video):
    cap = cv.VideoCapture(cv.samples.findFile(path_Video))
    no_Frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    print( 'Total number of frames is' , no_Frames )
 
    feature_params = dict( maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7 )
    color = np.random.randint(0, 255, (100, 3))

    ret, old_frame = cap.read()
    prevImg = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
    good_Feture = cv.goodFeaturesToTrack(prevImg, mask = None, **feature_params)
    mask = np.zeros_like(old_frame)
    dt = 100
    while(1):
        ret, frame = cap.read()
        if not ret:
            print('No frames grabbed!')
            break

        nextImg = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # code goes here
        p1, st, err = cv.calcOpticalFlowPyrLK(
            prevImg, nextImg, good_Feture, None,
            winSize=(15, 15), maxLevel=2,
            criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
        )

        if p1 is not None:
            good_new = p1[st==1]
            good_old = good_Feture[st==1]

        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = np.ravel(old)
            c, d = np.ravel(new)
            v_x = int((c - a)/dt)
            v_y = int((b - d)/dt)
            scale = 100
            v_x = int((c-a)/dt*scale)
            v_y = int((b-d)/dt*scale)

            mask = cv.arrowedLine(mask, (int(a),int(b)), (int(a)+v_x, int(b)+v_y), (255, 255, 255), 1)
        img = cv.add(frame, mask)

        cv.imshow('Result sparse optical flow', img)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break

        prevImg = nextImg.copy()
        good_Feture = good_new.reshape(-1, 1, 2)

    cv.destroyAllWindows()

sparse_optical("resources/vtest.avi")

Total number of frames is 795
No frames grabbed!


<h1 style="color:rgb(150,77,0)">Note about the output visualization</h1>

Now, let's use Dense Optical Flow. We will be utilizing the <i><b>Farneback Method</b></i>, one of the most popular implementations of Dense Optical Flow.

Dense Optical Flow computes the optical flow vector for every pixel in each frame, representing both the direction and magnitude for each pixel.

Since Dense Optical Flow provides a denser result, we use color coding for <i><b>better visualization</b></i>. We utilize <i><b>HSV planes</b></i> to display the results of Dense Optical Flow: the direction corresponds to the <i><b>Hue value</b></i> of the image, while the magnitude corresponds to the <i><b>Value plane</b></i>.

<a name="Apply-Farneback"></a><h3>Apply Farneback Optical Flow</h3>

<b>Exercise 1.2. [20 Points]</b>
   
<b>Goal:</b> The aim here is to apply the <i><b>Farneback Method</b></i>.

<b>Task</b>
<ul>
<li>Complete the function <i><b>dense_optical</b></i> below to compute dense optical flow.</li>
<li>The <i><b>Farneback Method</b></i> can be used from the OpenCV library.</li>
<li>The parameters of the <i><b>Farneback Method</b></i> are as follows: 
<ol>
  <li>parameters for computing the flow including the previous flow = None</li>
  <li>image scale for pyramid construction = 0.5</li>
  <li>number of pyramid layers = 3</li>
  <li>window size = 15</li>
 <li>number of iterations at each pyramid level = 3</li>
 <li>size of the pixel neighborhood for polynomial expansion = 5</li>
 <li>The standard deviation of the Gaussian = 1.2</li>
  <li>operation flags = 0</li>
</ol>

</ul>

<b>Hint</b>
<ul> 
<li>In the same manner, the function <i><b>dense_optical</b></i> uses the video <i><b>vtest.avi</b></i></li> 
<li>Again, the video <i><b>vtest.avi</b></i> is in the <i><b>resources</b></i> folder. <i><b>Therefore, you must make sure to include this video</b></i>.</li>
</ul>

In [14]:
def dense_optical(path_Video):
    
    cap = cv.VideoCapture(cv.samples.findFile(path_Video))
    no_Frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    print( 'Total number of frames is' , no_Frames )
    
    ret, prvs_frame1 = cap.read()
    prvs_frame = cv.cvtColor(prvs_frame1, cv.COLOR_BGR2GRAY)
    hsv = np.zeros_like(prvs_frame1)
    hsv[..., 1] = 255

    while(1):
        ret, next_frame2 = cap.read()
        frame_copy = next_frame2
        if not ret:
            print('No frames grabbed!')
            break
        next_frame = cv.cvtColor(next_frame2, cv.COLOR_BGR2GRAY)

        # The code goes here
        flow = cv.calcOpticalFlowFarneback(
            prvs_frame, next_frame, flow=None, 
            pyr_scale=0.5, levels=3, winsize=15, 
            iterations=3, poly_n=5, poly_sigma=1.2, flags=0
        )

        mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang*180/np.pi/2
        hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
        bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
        
        img = cv.add(next_frame2, bgr)
        cv.imshow('Result dense optical flow', img)        

        
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
            
        prvs_frame = next_frame
    cap.release()
    cv.destroyAllWindows()

dense_optical("resources/vtest.avi")

Total number of frames is 795
No frames grabbed!


<b>Exercise 1.3. [20 Points]</b>

<b>Question</b>
<ul>
<li>Dense optical flow methods, such as Farneback, provide flow vectors for each pixel. Why should we still use sparse optical flow methods, such as Lucas-Kanade? What are the advantages of sparse optical flow over dense optical flow?</li>    
</ul>

The explanation goes here:

<a name="Object-Tracking"></a><h2> Object Tracking</h2>

In the second part, we utilize the most popular technique in image tracking: <i><b>YOLOv8</b><i><i>. 

<span style="color:rgb(200,100,10)">More details can be found at [this link](https://docs.ultralytics.com/modes/track/)</span>.

<b>As we did in assignment 2, let's complete a couple of steps as follows:</b>
<ul> 
<li>We need to install the <i><b>ultralytics</b></i> package using this command: <i><b>pip install ultralytics</b>.</i>
<li>We need to import YOLO.</i>
</li> 

In [15]:
from ultralytics import YOLO

<a name=">Apply-Yolov8-for-object-tracking"></a><h3>Apply Yolov8 for object tracking</h3>

<b>Exercise 2.1. [20 Points]</b>
   
<b>Goal:</b> Let's apply the most popular tracker which is <i><b>YOLOv8</b></i>.

<b>Task</b>
<ul> 
<li>Complete the function <i><b>yolo_Track</b></i> below to apply the object tracking using <i><b>YOLOv8</b></i>.</li>
<li>The parameters of the <i><b>YOLO tracker</b></i> are as follows: 
<ol>
  <li>persist = True</li>
</ol>
</ul>

<b>Hint</b>
<ul> 
<li>The pre-trained model <i><b>yolov8n.pt</b></i> is utilized in this assignment.</li>
<li>In the same manner, the function <i><b>yolo_Track</b></i> uses the video <i><b>vtest.avi</b></i></li>
<li>Again, the video <i><b>vtest.avi</b></i> is in the <i><b>resources</b></i> folder. <i><b>Therefore, you must make sure to include this video</b></i>.</li> 
</ul>

In [16]:
# YOLO TRACKING
def yolo_Track(video_path, inter_Frame=5):
    model = YOLO("yolov8n.pt")
            
    cap = cv.VideoCapture(video_path)
    no_Frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    print( 'Total number of frames is' , no_Frames )
    
    while(1):
        ret, frame = cap.read()
        if not ret:
            print('No frames grabbed!')
            break
        
        # The code goes here
        results = model.track(source=frame, persist=True)

        frame_ = results[0].plot()
        
        cv.imshow('Result YOLO frame', frame_)

                        
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break

    cap.release()
    cv.destroyAllWindows()

yolo_Track("resources/vtest.avi")

Total number of frames is 795

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 24.5ms
Speed: 1.1ms preprocess, 24.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 24.5ms
Speed: 1.4ms preprocess, 24.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 22.2ms
Speed: 1.3ms preprocess, 22.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 22.3ms
Speed: 1.0ms preprocess, 22.3ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 21.5ms
Speed: 1.0ms preprocess, 21.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 23.0ms
Speed: 1.1ms preprocess, 23.0ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 20.5ms
Speed: 1.1

<b>Exercise 2.2. [20 Points]</b>

<b>Question</b>
<ul>
<li>What is the difference between Optical Flow and YOLO?</li>    
</ul>

The explanation goes here: